<a href="https://colab.research.google.com/github/Apekshagaonkar/dl_covid/blob/main/0.Dataset_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [3]:
%cd "/content/drive/My Drive/Covid19AI"

/content/drive/My Drive/Covid19AI


In [4]:
!cp "/content/drive/My Drive/Covid19AI/Jupyter_notebooks/BEASF.py" .

In [5]:
!cp "/content/drive/My Drive/Covid19AI/Jupyter_notebooks/visualization_tools.py" .

In [6]:
!pip3 install lime

     |████████████████████████████████| 276kB 7.8MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283845 sha256=fe711635f12c204e7b0d25bf31dba777cc2262f2cf02b818c19504cf52909ef7
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


In [7]:
import glob, numpy as np, cv2, matplotlib.pyplot as plt, copy, tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras import Model, callbacks as cb, optimizers
from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.layers import Dense, Dropout, Lambda, Input, Conv2D, Conv2DTranspose, concatenate, MaxPooling2D
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from skimage.filters import rank
from skimage.morphology import disk

from BEASF import BEASF
from visualization_tools import GradCAM

# read Data and convert to npy

In [8]:
def get_images(img_path): # FUNCTION FOR 1 FOLDER WITH ALL IMAGES MIXED WITH RESPECTIVE LABEL NAME
  normal_images = list()
  cap_images = list()
  covid_images = list()
  for img_name in glob.glob(pathname=img_path + '/*'):
    img = load_img(path=img_name, color_mode='grayscale')
    img = img_to_array(img=img, data_format='channels_last')
    if 'COVID' in img_name:
      covid_images.append(img)
    elif 'pneumonia' in img_name:
      cap_images.append(img)
    elif 'normal' in img_name:
      normal_images.append(img)
  normal_images = np.array(normal_images)
  cap_images = np.array(normal_images)
  covid_images = np.array(normal_images)

  normal_labels = [0 for _ in range(len(normal_images))]
  cap_labels = [1 for _ in range(len(cap_images))]
  covid_labels = [2 for _ in range(len(covid_images))]

  X = np.concatenate((covid_images, cap_images, normal_images))
  y = np.array(covid_labels + cap_labels + normal_labels)
  return X,y

def get_images(img_path): # FUNCTION FOR 1 FOLDER WITH FOLDERS WITH LABEL NAME AND RESPECTIVE IMAGES
  normal_images = list()
  cap_images = list()
  covid_images = list()
  print(glob.glob(pathname=img_path + '/*'))
  for folder_name in glob.glob(pathname=img_path + '/*'):
    print(folder_name)
    img_path =folder_name
    if 'covid' in folder_name:
      for img_name in glob.glob(pathname=img_path + '/*'):
        img = load_img(path=img_name, color_mode='grayscale')
        img = img_to_array(img=img, data_format='channels_last')
        covid_images.append(img)
    elif 'PNEUMONIA' in folder_name:
       for img_name in glob.glob(pathname=img_path + '/*'):
        img = load_img(path=img_name, color_mode='grayscale')
        img = img_to_array(img=img, data_format='channels_last')
        cap_images.append(img)
    elif 'NORMAL' in folder_name:
       for img_name in glob.glob(pathname=img_path + '/*'):
        img = load_img(path=img_name, color_mode='grayscale')
        img = img_to_array(img=img, data_format='channels_last')
        normal_images.append(img)
  normal_images = np.array(normal_images)
  cap_images = np.array(normal_images)
  covid_images = np.array(normal_images)

  normal_labels = [0 for _ in range(len(normal_images))]
  cap_labels = [1 for _ in range(len(cap_images))]
  covid_labels = [2 for _ in range(len(covid_images))]

  X = np.concatenate((covid_images, cap_images, normal_images))
  y = np.array(covid_labels + cap_labels + normal_labels)
  return X,y


In [9]:
def resize_images_of(X):
    X = np.array([cv2.resize(image, dsize=(320, 320), interpolation=cv2.INTER_CUBIC) for image in X])
    X = np.array([np.expand_dims(a=image, axis=-1) for image in X])
    X = X.astype(dtype=np.uint8)
    return X


def show_random_image_from(X):
    rnd_idx = np.random.choice(a=len(X), size=None)
    plt.imshow(X=X[rnd_idx].squeeze(), cmap='gray')
    plt.axis('off')
    plt.title(label='a random image from the dataset')
    plt.show()


def save_dataset(data, labels):
    print('number of total dataset images:', len(data))
    print('number of total dataset labels:', len(labels))
    print('dataset shape:', data.shape)
    print("export images as npy file? (y/n) : default = n need to remove commented code if needed to export")
    """if input() == 'y':
        np.save('./covid_cxr_dataset/cxr_samples.npy', arr=data)
        np.save('./covid_cxr_dataset/cxr_labels.npy', arr=labels)
    else:
        print('dataset exportation aborted.')"""
    print('saved as ./covid_cxr_dataset/cxr_samples.npy')
    print('saved as ./covid_cxr_dataset/cxr_labels.npy')

In [10]:
!ls

 0.Dataset_prep.ipynb	      Model_flow.ipynb
 1.Image_segmentation.ipynb   Pretrained_Model
 2.Training_model.ipynb      'Projects progress.gdoc'
 Additional		      __pycache__
 BEASF.py		      tmp.h5
 covid_cxr_dataset	     'v1.0_eps=099_valAcc=0.9672.hdf5'
 Documents		      visualization_tools.py
 Jupyter_notebooks


In [11]:
data_path = '/content/drive/My Drive/Covid19AI/covid_cxr_dataset/Training_data'

#X,y = get_images(img_path=data_path)


In [12]:
p2c = '/content/drive/My Drive/Covid19AI/covid_cxr_dataset/Training_data/covid19'
p2p = '/content/drive/My Drive/Covid19AI/covid_cxr_dataset/Training_data/PNEUMONIA'
p2n = '/content/drive/My Drive/Covid19AI/covid_cxr_dataset/Training_data/NORMAL'
normal_images = list()
cap_images = list()
covid_images = list()

In [ ]:
for img_name in glob.glob(pathname=p2c + '/*'):
        img = load_img(path=img_name, color_mode='grayscale')
        img = img_to_array(img=img, data_format='channels_last')
        covid_images.append(img)

In [ ]:
for img_name in glob.glob(pathname=p2p + '/*'):
        img = load_img(path=img_name, color_mode='grayscale')
        img = img_to_array(img=img, data_format='channels_last')
        cap_images.append(img)

In [ ]:
#normal_images = np.array(normal_images)
cap_images = np.array(normal_images)
covid_images = np.array(normal_images)

normal_labels = [0 for _ in range(len(normal_images))]
cap_labels = [1 for _ in range(len(cap_images))]
covid_labels = [2 for _ in range(len(covid_images))]

X = np.concatenate((covid_images, cap_images, normal_images))
y = np.array(covid_labels + cap_labels + normal_labels)

print(y)

In [ ]:
np.save('./covid_cxr_dataset/train_cxr_samples.npy', arr=X)
np.save('./covid_cxr_dataset/train_cxr_labels.npy', arr=y)

In [ ]:
X = resize_images_of(X=X)
show_random_image_from(X=X)
save_dataset(data=X, labels=y)

# Verifying the dataset preperation

In [ ]:
X = np.load(file='/content/drive/My Drive/Covid19AI/covid_cxr_dataset/cxr_samples.npy')
y = np.load(file='/content/drive/My Drive/Covid19AI/covid_cxr_dataset/cxr_labels.npy')

print('number of total dataset images:', len(X))
print('number of total dataset labels:', len(y))
print('dataset shape:', X.shape)

rnd_idx = np.random.choice(a=len(X), size=None)
plt.imshow(X=X[rnd_idx].squeeze(), cmap='gray')
plt.axis('off')
plt.title(label='a random image from the dataset')
plt.show()

In [ ]:
print(y)